# add title

## Import libraries

In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator

print("Tensorflow version %s" %tf.__version__)
print("Keras version %s" %keras.__version__)

Tensorflow version 2.4.1
Keras version 2.4.3


## Load data

In [ ]:
datadir = '/dataset/'
trainingset = datadir+'/train/'
testset = datadir + '/test/'

batch_size = 16
train_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    directory=trainingset,
    target_size=(118, 224), #TODO
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True)

test_datagen = ImageDataGenerator(
    rescale = 1. / 255)

test_generator = test_datagen.flow_from_directory(
    directory=testset,
    target_size=(118, 224),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

num_samples = train_generator.n
num_classes = train_generator.num_classes
input_shape = train_generator.image_shape

classnames = [k for k,v in train_generator.class_indices.items()]

print("Image input %s" %str(input_shape))
print("Classes: %r" %classnames)

print('Loaded %d training samples from %d classes.' %(num_samples,num_classes))
print('Loaded %d test samples from %d classes.' %(test_generator.n,test_generator.num_classes))

## Model

In [9]:
from keras.models import Sequential
from keras.layers import SeparableConv2D, ZeroPadding2D, Activation, Dropout, Dense, \
                            Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras import Input, optimizers

def Net(input_shape, num_classes, dilated_kernel, dilation, dilated_padding):

    model = Sequential()
    
    # input layer
    model.add(Input(shape=(256,256,1)))#TODO ADD REAL SIZE
    
    # DWS-CNN layer 1
    model.add(ZeroPadding2D(padding=(2)))
    # use valid padding since padding is introduced before since it has a special form
    model.add(SeparableConv2D(256,kernel_size=(5,5), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation before passing it to the next layer
    model.add(BatchNormalization())
    # Pooling
    model.add(MaxPooling2D(pool_size=(1,5), strides=(1,5), padding='valid'))
    # Dropout
    model.add(Dropout(0.25))
    
    # DWS-CNN layer 2
    model.add(ZeroPadding2D(padding=(2, 2)))
    # use valid padding since padding is introduced before since it has a special form
    model.add(SeparableConv2D(256,kernel_size=(5,5), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation before passing it to the next layer
    model.add(BatchNormalization())
    # Pooling
    model.add(MaxPooling2D(pool_size=(1,4), strides=(1,4), padding='valid'))
    # Dropout
    model.add(Dropout(0.25))
    
    # DWS-CNN layer 3
    model.add(ZeroPadding2D(padding=(2, 2)))
    # use valid padding since padding is introduced before since it has a special form
    model.add(SeparableConv2D(256,kernel_size=(5,5), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation before passing it to the next layer
    model.add(BatchNormalization())
    # Pooling
    model.add(MaxPooling2D(pool_size=(1,2), strides=(1,2), padding='valid'))
    # Dropout
    model.add(Dropout(0.25))
    
    # DIL-CNN 
    model.add(ZeroPadding2D(padding=(0, dilated_padding*dilation)))
    model.add(Conv2D(256, kernel_size=dilated_kernel, dilation_rate=(1,dilation)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    # classifier layer
    model.add(Dense(num_classes))
    
    # model compilation for training
    adam = optimizers.Adam()
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy', 'precision', 'recall'])
    
    return model


# create the model
input_shape = (16, 16, 16, 1)
num_classes = 16
dilated_kernel = (3,3)
dilation = (10)
dilated_padding = 2
model = Net(input_shape,num_classes,dilated_kernel,dilation,dilated_padding)
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_22 (ZeroPaddi (None, 260, 260, 1)       0         
_________________________________________________________________
separable_conv2d (SeparableC (None, 256, 256, 256)     537       
_________________________________________________________________
activation_20 (Activation)   (None, 256, 256, 256)     0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 256, 256, 256)     1024      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 256, 51, 256)      0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 256, 51, 256)      0         
_________________________________________________________________
zero_padding2d_23 (ZeroPaddi (None, 260, 55, 256)     

## Train the model

In [ ]:
steps_per_epoch=train_generator.n//train_generator.batch_size
val_steps=test_generator.n//test_generator.batch_size+1
epochs = 1
try:
    history = model.fit(train_generator, epochs=epochs, verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=test_generator,\
                    validation_steps=val_steps)
except KeyboardInterrupt:
    pass